# Semana 5

## Threading

### Introducción

En esta semana, vamos a aprender sobre threading en Python. Threading es una técnica que permite a un programa realizar múltiples tareas simultáneamente. En Python, el módulo `threading` proporciona una forma sencilla de crear y administrar hilos.